### Nombre: Elena Jaramillo Rosado

## Primer Clasificador TFM

Este primer clasificador toma los datos del dataset __dataset_translated.csv__ y predice a partir de las columnas 'bio' e 'interest' si el género ('gender') es Hombre (**Man**) o Mujer (**Woman**). El objetivo de este clasificador es el de obtener un dataset resultante en el que el número de registros donde gender = 'NotSpecified' es similar al número de registros donde gender = 'Woman' o 'Man' 

El procedimiento que se va a seguir es de self-training. Va a consistir de los siguientes pasos:

- Se entrena un modelo base (en este caso, se usará Regresión Logística) usando el conjunto de datos etiquetados disponible.

- El modelo entrenado se usa para predecir las etiquetas de los datos no etiquetados.

- Se seleccionan registros del conjunto no etiquetado cuyos resultados predichos tienen una alta confianza.

- Los registros seleccionados se agregan al conjunto de datos etiquetado.
    
- Se vuelve a entrenar el modelo con el conjunto etiquetado expandido. 

Este proceso se hará de forma iterativa hasta que se lleguen a la mitad de los datos etiquetados o no existan predicciones con la suficiente confianza.

In [ ]:
from utils.Preprocessing import Preprocessing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [27]:
# Ruta al dataset
path_df = './datasets/dataset_translated.csv'

df = pd.read_csv(path_df, sep = ',', header = 0)
print(df.head(10))

    age                                                bio  \
0  23.0  Do I want it or do I want techno?|The one with...   
1  27.0                    with something to do tomorrow.|   
2  20.0  a few joints and a beer or are we going to a r...   
3  22.0  nice but clumsy.|funny but stubborn.|clown but...   
4  26.0                               something different!   
5  21.0  ||:woman_police_officer_light_skin_tone::blue_...   
6  18.0  If you are a fascist, go back to where you cam...   
7  18.0                    ::anatomical_heart:|:Venezuela:   
8  34.0  trans girl :transgender_flag: |if you like it;...   
9  23.0  fun girl; sincere, passionate about music | co...   

                           job                          school  \
0             ['NotSpecified']                             NaN   
1             ['NotSpecified']                             NaN   
2                  ['ketalal']                             NaN   
3             ['NotSpecified']                       

In [28]:
# Preprocessing() es una clase que he creado para englobar acciones de preprocesamiento.

preprocessing = Preprocessing()

df['bio'] = df['bio'].apply(preprocessing.lowercase).apply(preprocessing.remove_vertical_bars)
df['interest_text'] = df['interest_text'].apply(preprocessing.lowercase).apply(preprocessing.remove_vertical_bars)
df['gender'] = df['gender'].str.strip("[]'")

In [29]:
df['gender'].value_counts()

gender
NotSpecified            3054
Woman                    445
Man                      442
Mujer trans                3
Género no binario          1
idea                       1
Pareja                     1
Trans man                  1
fluid                      1
ゲイ                         1
Transgender Woman          1
Trans woman                1
Bisexual                   1
Trans                      1
Cupcake de frambuesa       1
trans                      1
Name: count, dtype: int64

Vemos que las clases mayoritarias del atributo 'gender' son NotSpecified, Woman y Man. Para facilitarnos la tarea, vamos a quedarnos con los registros que tengan estos valores en 'gender'. Debido a que el resto de clases son muy poco frecuentes (en total, suman 15), vamos a decidir prescindir de ellos.

In [30]:
# Separar los datos etiquetados (Man, Woman) y no etiquetados (NotSpecified)
labeled_data = df[df['gender'].isin(['Woman', 'Man'])]
unlabeled_data = df[df['gender'] == 'NotSpecified']

In [31]:
# Vectorizar los datos de 'bio' e 'interest'
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
labeled_vectors = vectorizer.fit_transform(labeled_data['bio'] + ' ' + labeled_data['interest_text'])
unlabeled_vectors = vectorizer.transform(unlabeled_data['bio'] + ' ' + unlabeled_data['interest_text'])

In [32]:
# Etiquetas para los datos etiquetados
y_labeled = labeled_data['gender']

In [33]:
# Paso 4: Entrenar un modelo de regresión logística. Dividimos los datos de train y test en una proporcioón 70-30.
X_train, X_test, y_train, y_test = train_test_split(labeled_vectors, y_labeled, test_size=0.3, random_state=42)
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [34]:
# Evaluar el modelo en los datos de test
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Man       0.78      0.69      0.73       138
       Woman       0.70      0.79      0.74       129

    accuracy                           0.74       267
   macro avg       0.74      0.74      0.74       267
weighted avg       0.74      0.74      0.74       267



In [35]:
#Self-Training con iteraciones
target_labeled_size = len(df)/2
confidence_threshold = 0.83  # Umbral de confianza
iteration = 0

while len(labeled_data) < target_labeled_size:
    print(f"Iteración {iteration + 1}")

    # Predecir probabilidades para los datos no etiquetados
    probs = model.predict_proba(unlabeled_vectors)
    predicted_labels = model.predict(unlabeled_vectors)

    # Seleccionar predicciones con alta confianza
    confident_indices = np.max(probs, axis=1) >= confidence_threshold
    confident_unlabeled_data = unlabeled_data[confident_indices]
    if confident_unlabeled_data.empty:
        print("No se encontraron predicciones confiables. Deteniendo iteraciones.")
        break

    # Añadir datos confiables al conjunto de entrenamiento
    confident_vectors = unlabeled_vectors[confident_indices]
    confident_labels = predicted_labels[confident_indices]

    if iteration == 0:
        labeled_vectors = np.vstack([labeled_vectors.toarray(), confident_vectors.toarray()])
        y_labeled= np.concatenate([y_labeled, confident_labels])
    else:
        labeled_vectors_resampled = np.vstack([labeled_vectors, confident_vectors.toarray()])
        y_labeled_resampled = np.concatenate([y_labeled, confident_labels])


    # Actualizar los datos etiquetados y no etiquetados
    labeled_data = pd.concat([labeled_data, confident_unlabeled_data.assign(gender=confident_labels)])
    unlabeled_vectors = unlabeled_vectors[~confident_indices]
    unlabeled_data = unlabeled_data[~confident_indices]

    # Reentrenar el modelo con el conjunto ampliado
    model.fit(labeled_vectors, y_labeled)
    iteration += 1

    # Evaluar el modelo con el conjunto de prueba
    y_pred = model.predict(X_test)
    print(f"Reporte después de la iteración {iteration}:")
    print(classification_report(y_test, y_pred))

    # Verificar si hemos alcanzado la mitad del dataset etiquetado
    if len(labeled_data) >= target_labeled_size:
        print("Se ha alcanzado la mitad del dataset etiquetado.")
        break


Iteración 1
Reporte después de la iteración 1:
              precision    recall  f1-score   support

         Man       0.91      0.90      0.90       138
       Woman       0.89      0.90      0.90       129

    accuracy                           0.90       267
   macro avg       0.90      0.90      0.90       267
weighted avg       0.90      0.90      0.90       267

Iteración 2
Reporte después de la iteración 2:
              precision    recall  f1-score   support

         Man       0.91      0.90      0.90       138
       Woman       0.89      0.90      0.90       129

    accuracy                           0.90       267
   macro avg       0.90      0.90      0.90       267
weighted avg       0.90      0.90      0.90       267

Iteración 3
No se encontraron predicciones confiables. Deteniendo iteraciones.


In [36]:
# Distribución final de géneros
print("Distribución final de géneros etiquetados:")
print(labeled_data['gender'].value_counts())

Distribución final de géneros etiquetados:
gender
Man      954
Woman    551
Name: count, dtype: int64


Finalmente, hemos obtenido en total 1505 registros etiquetados como 'Man' o 'Woman', frente a los casi 900 del principio. Eportamos el resultado a otro .csv que se usará en segundo clasifcador.

In [37]:
final_dataset = pd.concat([labeled_data, unlabeled_data])
final_dataset.to_csv('./datasets/dataset_more_labeled.csv', index = False)